<a href="https://colab.research.google.com/github/rohitnikam1/Basic-Financial-Analysis/blob/master/1_Means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measures of Central Tendency

In this notebook we will discuss ways to summarize a set of data using a single number. The goal is to capture information about the distribution of data.

## Arithmetic Mean

The arithmetic mean is used very frequently to summarize numerical data, and is usually the one assumed to be meant by the word "average." It is defined as the sum of the observations divided by the number of observations:

\begin{align}
\hat{\mu} = \frac{\sum_i^n X_i}{n}
\end{align}
where $X_i$ are sample observations.

In [56]:
!pip3 install alpha_vantage

     |████████████████████████████████| 1.3 MB 10.3 MB/s 
     |████████████████████████████████| 271 kB 53.8 MB/s 
     |████████████████████████████████| 160 kB 72.4 MB/s 


In [58]:
from alpha_vantage.timeseries import TimeSeries
import time
api_key = '60PFJXDG66HJDDXJ'

In [ ]:
# Two useful statistical libraries
import scipy.stats as stats
import numpy as np
import pandas as pd

In [ ]:
# We'll use these two data sets as examples
x1 = [1, 2, 2, 3, 4, 5, 5, 7]
x2 = x1 + [100]

print(f'Mean of x1: {sum(x1)}/{len(x1)} = {np.mean(x1)}')
print(f'Mean of x2: {sum(x2)}/{len(x2)} = {np.mean(x2)}')

Mean of x1: 29/8 = 3.625
Mean of x2: 129/9 = 14.333333333333334


We can also define a weighted arithmetic mean, which is useful for explicitly specifying the number of times each observation should be counted. For instance, in computing the average value of a portfolio, it is more convenient to say that 70% of your stocks are of type X rather than making a list of every share you hold.

The weighted arithmetic mean is defined as

\begin{align}
\sum_i^n w_i X_i
\end{align}

where $\sum_i^n w_i = 1$. For normal arithmetic mean, $w_i = 1/n \quad \forall \quad i$.

## Median

The median of a set of data is the number which appears in the middle of the list when it is sorted in increasing or decreasing order. When we have an odd number $n$ of data points, this is simply the value in position $(n+1)/2$. When we have an even number of data points, the list splits in half and there is no item in the middle; so we define the median as the average of the values in positions $n/2$ and $(n+2)/2$.

The median is less affected by extreme values in the data than the arithmetic mean. It tells us the value that splits the data set in half, but not how much smaller or larger the other values are.

In [ ]:
print(f'Median of x1: {np.median(x1)}')
print(f'Median of x2: {np.median(x2)}')

Median of x1: 3.5
Median of x2: 4.0


# Mode

The mode is the most frequently occuring value in a data set. It can be applied to non-numerical data, unlike the mean and the median. One situation in which it is useful is for data whose possible values are independent. For example, in the outcomes of a weighted die, coming up 6 often does not mean it is likely to come up 5; so knowing that the data set has a mode of 6 is more useful than knowing it has a mean of 4.5.

In [ ]:
# Scipy has a built-in mode function, but it will return exactly one value  even if two values occur the same number of times, or if no value appears more than once
print(f'One mode of x1: {stats.mode(x1)[0][0]}')

# So we will write our own.
# We will need the following function which sorts the dictionary by values
def sort_dict(d):
  keys = sorted(d, reverse=True, key=d.get)
  return {key:d[key] for key in keys}

from collections import defaultdict

def mode(l):
  # list l has all elements of the same datatype
  t = type(l[0])
  d = defaultdict(t)
  for item in l:
    d[item] += 1
  sorted_ = sort_dict(d)
  return dict(filter(lambda x: x[1]>1, sorted_.items()))
  

print(f'All modes of [1,1,5,2,2,6,6,6]: {mode([1,1,5,2,2,6,6,6])}')


One mode of x1: 2
All modes of [1,1,5,2,2,6,6,6]: {6: 3, 1: 2, 2: 2}


For data that can take on many different values, such as returns data, there may not be any values that appear more than once. In this case we can bin values, like we do when constructing a histogram, and then find the mode of the data set where each value is replaced with the name of its bin. That is, we find which bin elements fall into most often.

In [68]:
ts = TimeSeries(key=api_key, output_format='pandas')
data, metadata = ts.get_intraday(symbol='MSFT', interval='1min', outputsize='full')
data = data['4. close']
returns = data.pct_change()[1:]

In [69]:
returns

date
2021-10-08 19:55:00   -0.000645
2021-10-08 19:54:00    0.000679
2021-10-08 19:45:00    0.000000
2021-10-08 19:19:00    0.000340
2021-10-08 18:36:00    0.000068
                         ...   
2021-09-27 04:25:00    0.000100
2021-09-27 04:11:00    0.000000
2021-09-27 04:06:00    0.000534
2021-09-27 04:05:00    0.000534
2021-09-27 04:02:00    0.001001
Name: 4. close, Length: 6252, dtype: float64

In [70]:
# Since all of the returns are distinct, we use a frequency distribution to get an alternative mode.
# np.histogram returns the frequency distribution over the bins as well as the endpoints of the bins
count, bins = np.histogram(returns, 20) # separate data in 20 bins. To specify bins use `bins =[]` argument.
maxfreq = max(count)
# Find all of the bins that are hit with frequency maxfreq, then print the intervals corresponding to them
print(f'Mode of bins: {[(bins[i], bins[i+1]) for i,j in enumerate(count) if j == maxfreq]}')

Mode of bins: [(-6.92043940563828e-05, 0.0009590641122139698)]


In [ ]:
series = pd.Series([0.0,950.0,-70.0,812.0,0.0,-90.0,0.0,0.0,-90.0,0.0,-64.0,208.0,0.0,-90.0,0.0,-80.0,0.0,0.0,-80.0,-48.0,840.0,-100.0,190.0,130.0,-100.0,-100.0,0.0,-50.0,0.0,-100.0,-100.0,0.0,-90.0,0.0,-90.0,-90.0,63.0,-90.0,0.0,0.0,-90.0,-80.0,0.0,])


Mode of bins: [(-100.0, -47.5)]


# Geometric Mean

While the arithmetic mean averages using addition, the geometric mean uses multiplication:$$ G = \sqrt[n]{X_1X_1\ldots X_n} $$

for observations $X_i \geq 0$. We can also rewrite it as an arithmetic mean using logarithms:$$ \ln G = \frac{\sum_{i=1}^n \ln X_i}{n} $$

**The geometric mean is always less than or equal to the arithmetic mean (when working with nonnegative observations), with equality only when all of the observations are the same**

In [ ]:
# Use scipy's gmean function to compute the geometric mean
print(f'Geometric mean of x1: {stats.gmean(x1)}')
print(f'Geometric mean of x2: {stats.gmean(x2)}')

Geometric mean of x1: 3.0941040249774403
Geometric mean of x2: 4.552534587620071


What if we want to compute the geometric mean when we have negative observations? This problem is easy to solve in the case of asset returns, where our values are always at least $-1$. We can add 1 to a return $R_t$ to get $1 + R_t$, which is the ratio of the price of the asset for two consecutive periods (as opposed to the percent change between the prices, $R_t$). This quantity will always be nonnegative. So we can compute the geometric mean return,$$ R_G = \sqrt[T]{(1 + R_1)\ldots (1 + R_T)} - 1$$

In [71]:
# Add 1 to every value in the returns array and then compute R_G
ratios = returns + 1
R_G = stats.gmean(ratios) - 1
print(f'Geometric mean of returns: {R_G}')

Geometric mean of returns: 2.9433199395523246e-06


The geometric mean is defined so that if the rate of return over the whole time period were constant and equal to $R_G$, the final price of the security would be the same as in the case of returns $R_1, \ldots, R_T$.

In [73]:
T = len(returns)
init_price = data[0]
final_price = data[T]
print(f'Initial price: {init_price}')
print(f'Final price: {final_price}')
print(f'Final price as computed with R_G: {init_price*(1 + R_G)**T}')

Initial price: 294.53
Final price: 300.0
Final price as computed with R_G: 300.0000000001892


## Harmonic Mean

The harmonic mean is less commonly used than the other types of means. It is defined as$$ H = \frac{n}{\sum_{i=1}^n \frac{1}{X_i}} $$

As with the geometric mean, we can rewrite the harmonic mean to look like an arithmetic mean. The reciprocal of the harmonic mean is the arithmetic mean of the reciprocals of the observations:$$ \frac{1}{H} = \frac{\sum_{i=1}^n \frac{1}{X_i}}{n} $$

**The harmonic mean for nonnegative numbers $X_i$ is always at most the geometric mean (which is at most the arithmetic mean), and they are equal only when all of the observations are equal.**

In [75]:
print(f'Harmonic mean of x1: {stats.hmean(x1)}')
print(f'Harmonic mean of x2: {stats.hmean(x2)}')

Harmonic mean of x1: 2.5590251332825593
Harmonic mean of x2: 2.869723656240511


The harmonic mean can be used when the data can be naturally phrased in terms of ratios. For instance, in the dollar-cost averaging strategy, a fixed amount is spent on shares of a stock at regular intervals. The higher the price of the stock, then, the fewer shares an investor following this strategy buys. The average (arithmetic mean) amount they pay for the stock is the harmonic mean of the prices.

# Point Estimates Can Be Deceiving

Means by nature hide a lot of information, as they collapse entire distributions into one number. As a result often 'point estimates' or metrics that use one number, can disguise large programs in your data. You should be careful to ensure that you are not losing key information by summarizing your data, and you should rarely, if ever, use a mean without also referring to a measure of spread.

## Underlying Distribution Can be Wrong

Even when you are using the right metrics for mean and spread, they can make no sense if your underlying distribution is not what you think it is. For instance, using standard deviation to measure frequency of an event will usually assume normality. Try not to assume distributions unless you have to, in which case you should rigourously check that the data do fit the distribution you are assuming.

## References

- "Quantitative Investment Analysis", by DeFusco, McLeavey, Pinto, and Runkle